## Import and load

In [1]:
import pandas as pd
from T_data_loader import *

In [2]:
#Names of columns for the data CMU
NAMES_MOVIES = ['Wikipedia_movie_ID','Freebase_movie_ID','Movie_name','Movie_release_date','Movie_box_office_revenue','Movie_runtime','Movie_languages','Movie_countries','Movie_genres']

In [3]:
df_CMU_movies = csv_loader('movie.metadata', 'CMU', column_names=NAMES_MOVIES)

In [4]:
df_IMDb = pd.read_pickle('IMDb/IMDb.pkl')

## Trying to get an unique index

In [5]:
df_CMU_movies["Movie_release_date"]=pd.to_datetime(df_CMU_movies["Movie_release_date"], format='mixed', errors='coerce').dt.year.astype('Int64')

In [7]:
df_IMDb['realease_date']=pd.to_datetime(df_IMDb['realease_date'], format='mixed', errors='coerce').dt.year.astype('Int64')

In [8]:
df_CMU_movies_modified_title = df_CMU_movies.Movie_name.str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True).str.lower()
df_CMU_movies["modified_title"]=df_CMU_movies_modified_title

In [9]:
df_IMDb_modified_title = df_IMDb.Primary_title.str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True).str.lower()
df_IMDb["modified_title"]=df_IMDb_modified_title

In [10]:
df_IMDb["runtime"] = pd.to_numeric(df_IMDb["runtime"], errors='coerce').round(0).astype('Int64')
df_CMU_movies["Movie_runtime"] = pd.to_numeric(df_CMU_movies["Movie_runtime"], errors='coerce').round(0).astype('Int64')

In [15]:
new_id = df_CMU_movies.Movie_release_date.astype(str)+df_CMU_movies.Movie_runtime.astype(str)+df_CMU_movies.modified_title

In [16]:
df_CMU_movies_newind = df_CMU_movies.copy()
df_CMU_movies_newind.index = new_id
df_CMU_movies_newind.index.is_unique

False

In [17]:
df_CMU_movies_newind.shape

(81741, 10)

In [18]:
df_CMU_movies_newind.index.value_counts()

<NA><NA>hero                    3
<NA><NA>vengeance               3
<NA><NA>wanted                  3
<NA><NA>busybuddies             2
195186littlebighorn             2
                               ..
1990100spacedinvaders           1
194355frontierfury              1
194659theflyingserpent          1
1956105janzizka                 1
200286thesorcerersapprentice    1
Name: count, Length: 81687, dtype: int64

In [52]:
df_CMU_movies.Movie_release_date.value_counts()

Movie_release_date
1970    74838
Name: count, dtype: Int64

In [22]:
new_id2 = df_CMU_movies.Movie_release_date.astype(str)+df_CMU_movies.modified_title

In [23]:
df_CMU_movies_newind2 = df_CMU_movies.copy()
df_CMU_movies_newind2.index = new_id2
df_CMU_movies_newind2.index.is_unique

False